data/data.csv
有重複資料
562 => 522

In [1]:
import pandas as pd
import torch
import torch.optim as optim

ModuleNotFoundError: No module named 'pandas'

In [6]:
df = pd.read_csv("data/data.csv")

In [9]:
df.count()

sublocation    562
Seq            562
dtype: int64

In [11]:
ben_df = pd.read_csv("data/lncLocPred/ben-sublocation-seq.csv")
ind_df = pd.read_csv("data/lncLocPred/ind-sublocation-seq.csv")

In [12]:
print(ben_df.count())
print(ben_df.drop_duplicates().count())
print(ind_df.count())
print(ind_df.drop_duplicates().count())

sublocation    225
Seq            225
dtype: int64
sublocation    225
Seq            225
dtype: int64
sublocation    337
Seq            337
dtype: int64
sublocation    297
Seq            297
dtype: int64


In [ ]:
print(ind_df.value_counts())

In [10]:
df.drop_duplicates().count()

sublocation    522
Seq            522
dtype: int64

In [5]:
df1.to_csv("data/data.csv", mode = 'a', header = False, index = False)